In [24]:
include("../scripts/robot.jl")

In [25]:
using Distributions
using LinearAlgebra

In [26]:
mutable struct Particle
    pose
end

In [27]:
function motion_update(self::Particle, nu, omega, time, noise_rate_pdf)
    ns = rand(noise_rate_pdf)
    noised_nu = nu + ns[1]*sqrt(abs(nu)/time) + ns[2]*sqrt(abs(omega)/time)
    noised_omega = omega + ns[3]*sqrt(abs(nu)/time) + ns[4]*sqrt(abs(omega)/time)
    self.pose = state_transition(IdealRobot([]), noised_nu, noised_omega, time, self.pose)
end

motion_update (generic function with 2 methods)

In [28]:
function observation_update(self::Particle, observation)
    print(observation)
end

observation_update (generic function with 2 methods)

In [29]:
mutable struct Mcl
    particles
    motion_noise_rate_pdf
end

In [30]:
function Mcl_(init_pose, num, motion_noise_stds=Dict("nn"=>0.19, "no"=>0.001, "on"=>0.13, "oo"=>0.2))
    self = Mcl(ntuple(x->nothing, fieldcount(Mcl))...)
    self.particles =[Particle(init_pose) for i in 1:num]
    v = motion_noise_stds
    c = Diagonal([v["nn"]^2, v["no"]^2, v["on"]^2, v["oo"]^2])
    self.motion_noise_rate_pdf = MvNormal(zeros(4), c)
    return self
end

Mcl_ (generic function with 2 methods)

In [31]:
function motion_update(self::Mcl, nu, omega, time)
    for p in self.particles
        motion_update(p, nu, omega, time, self.motion_noise_rate_pdf)
    end
end

motion_update (generic function with 2 methods)

In [40]:
function observation_update(self::Mcl, observation)
    for p in self.particles
        observation_update(p, observation)
    end
end

observation_update (generic function with 2 methods)

In [41]:
function draw(self::Mcl, ax, elems)
    xs = [p.pose[1] for p in self.particles]
    ys = [p.pose[2] for p in self.particles]
    vxs = [cos(p.pose[3]) for p in self.particles]
    vys = [sin(p.pose[3]) for p in self.particles]
    elems = vcat(elems, ax.quiver(xs, ys, vxs, vys, color="blue", alpha=0.5))
end

draw (generic function with 7 methods)

In [42]:
mutable struct EstimationAgent <: AbstractAgent
    nu
    omega
    estimator
    time_interval
    prev_nu
    prev_omega
end

In [43]:
function  EstimationAgent_(time_interval, nu, omega,estimator)
    self = EstimationAgent(ntuple(x->nothing, fieldcount(EstimationAgent))...)
    self.nu = nu
    self.omega = omega
    self.estimator = estimator
    self.time_interval = time_interval
    self.prev_nu = 0.0
    self.prev_omega = 0.0
    return self
end

EstimationAgent_ (generic function with 1 method)

In [48]:
function decision(self::EstimationAgent, observation=nothing)
    motion_update(self.estimator, self.prev_nu, self.prev_omega, self.time_interval)
    self.prev_nu, self.prev_omega = self.nu, self.omega
    observation_update(self.estimator,observation)
    return self.nu, self.omega
end

decision (generic function with 4 methods)

In [49]:
function draw(self::EstimationAgent, ax, elems)
    draw(self.estimator, ax, elems)
end

draw (generic function with 7 methods)

In [50]:
function trial()
    time_interval = 1.0
    world = World_(30, time_interval, true)
    
    m = Map()
    for ln in [[-4.0, 2.0], [2.0, -3.0], [3.0, 3.0]]
        append_landmark(m, Landmark(ln))
    end
    append(world, m)
    
    initial_pose = [0.0, 0.0, 0.0]
    estimator = Mcl_(initial_pose, 100)
    a = EstimationAgent_(time_interval, 0.2, 10.0/180*pi, estimator)
    r = Robot_(initial_pose, sensor=Camera_(m), agent=a, color="red")
    append(world, r)
    draw(world)
    showanim("test.mp4")
end

trial (generic function with 1 method)

In [51]:
trial()

Any[([3.6448098151886605, -0.8367659489234014], 1), ([4.2414243792035835, 0.7978668538817779], 2)]Any[([3.6448098151886605, -0.8367659489234014], 1), ([4.2414243792035835, 0.7978668538817779], 2)]Any[([3.6448098151886605, -0.8367659489234014], 1), ([4.2414243792035835, 0.7978668538817779], 2)]Any[([3.6448098151886605, -0.8367659489234014], 1), ([4.2414243792035835, 0.7978668538817779], 2)]Any[([3.6448098151886605, -0.8367659489234014], 1), ([4.2414243792035835, 0.7978668538817779], 2)]Any[([3.6448098151886605, -0.8367659489234014], 1), ([4.2414243792035835, 0.7978668538817779], 2)]Any[([3.6448098151886605, -0.8367659489234014], 1), ([4.2414243792035835, 0.7978668538817779], 2)]Any[([3.6448098151886605, -0.8367659489234014], 1), ([4.2414243792035835, 0.7978668538817779], 2)]Any[([3.6448098151886605, -0.8367659489234014], 1), ([4.2414243792035835, 0.7978668538817779], 2)]Any[([3.6448098151886605, -0.8367659489234014], 1), ([4.2414243792035835, 0.7978668538817779], 2)]Any[([3.644809815188

[([4.615088786718202, 0.780215609653742], 2)]Any[([4.615088786718202, 0.780215609653742], 2)]Any[([4.615088786718202, 0.780215609653742], 2)]Any[([4.615088786718202, 0.780215609653742], 2)]Any[([4.615088786718202, 0.780215609653742], 2)]Any[([4.615088786718202, 0.780215609653742], 2)]Any[([4.615088786718202, 0.780215609653742], 2)]Any[([4.615088786718202, 0.780215609653742], 2)]Any[([4.615088786718202, 0.780215609653742], 2)]Any[([4.615088786718202, 0.780215609653742], 2)]Any[([4.615088786718202, 0.780215609653742], 2)]Any[([4.615088786718202, 0.780215609653742], 2)]Any[([4.615088786718202, 0.780215609653742], 2)]Any[([4.615088786718202, 0.780215609653742], 2)]Any[([4.615088786718202, 0.780215609653742], 2)]Any[([4.615088786718202, 0.780215609653742], 2)]Any[([4.615088786718202, 0.780215609653742], 2)]Any[([4.615088786718202, 0.780215609653742], 2)]Any[([4.615088786718202, 0.780215609653742], 2)]Any[([4.615088786718202, 0.780215609653742], 2)]Any[([4.615088786718202, 0.780215609653742]

[([3.9245499365257226, 0.2414415115970055], 2)]Any[([3.9245499365257226, 0.2414415115970055], 2)]Any[([3.9245499365257226, 0.2414415115970055], 2)]Any[([3.9245499365257226, 0.2414415115970055], 2)]Any[([3.9245499365257226, 0.2414415115970055], 2)]Any[([3.9245499365257226, 0.2414415115970055], 2)]Any[([3.9245499365257226, 0.2414415115970055], 2)]Any[([3.9245499365257226, 0.2414415115970055], 2)]Any[([3.9245499365257226, 0.2414415115970055], 2)]Any[([3.9245499365257226, 0.2414415115970055], 2)]Any[([3.9245499365257226, 0.2414415115970055], 2)]Any[([3.9245499365257226, 0.2414415115970055], 2)]Any[([3.9245499365257226, 0.2414415115970055], 2)]Any[([3.9245499365257226, 0.2414415115970055], 2)]Any[([3.9245499365257226, 0.2414415115970055], 2)]Any[([3.9245499365257226, 0.2414415115970055], 2)]Any[([3.9245499365257226, 0.2414415115970055], 2)]Any[([3.9245499365257226, 0.2414415115970055], 2)]Any[([3.9245499365257226, 0.2414415115970055], 2)]Any[([3.9245499365257226, 0.2414415115970055], 2)]Any

[([2.716684288151678, -0.15200832275633125], 2)]Any[([2.716684288151678, -0.15200832275633125], 2)]Any[([2.716684288151678, -0.15200832275633125], 2)]Any[([2.716684288151678, -0.15200832275633125], 2)]Any[([2.716684288151678, -0.15200832275633125], 2)]Any[([2.716684288151678, -0.15200832275633125], 2)]Any[([2.716684288151678, -0.15200832275633125], 2)]Any[([2.716684288151678, -0.15200832275633125], 2)]Any[([2.716684288151678, -0.15200832275633125], 2)]Any[([2.716684288151678, -0.15200832275633125], 2)]Any[([2.716684288151678, -0.15200832275633125], 2)]Any[([2.716684288151678, -0.15200832275633125], 2)]Any[([2.716684288151678, -0.15200832275633125], 2)]Any[([2.716684288151678, -0.15200832275633125], 2)]Any[([2.716684288151678, -0.15200832275633125], 2)]Any[([2.716684288151678, -0.15200832275633125], 2)]Any[([2.716684288151678, -0.15200832275633125], 2)]Any[([2.716684288151678, -0.15200832275633125], 2)]Any[([2.716684288151678, -0.15200832275633125], 2)]Any[([2.716684288151678, -0.152008

[([3.2445719908771724, -0.5506901096452796], 2)]Any[([3.2445719908771724, -0.5506901096452796], 2)]Any[([3.2445719908771724, -0.5506901096452796], 2)]Any[([3.2445719908771724, -0.5506901096452796], 2)]Any[([3.2445719908771724, -0.5506901096452796], 2)]Any[([3.2445719908771724, -0.5506901096452796], 2)]Any[([3.2445719908771724, -0.5506901096452796], 2)]Any[([3.2445719908771724, -0.5506901096452796], 2)]Any[([3.2445719908771724, -0.5506901096452796], 2)]Any[([3.2445719908771724, -0.5506901096452796], 2)]Any[([3.2445719908771724, -0.5506901096452796], 2)]Any[([3.2445719908771724, -0.5506901096452796], 2)]Any[([3.2445719908771724, -0.5506901096452796], 2)]Any[([3.2445719908771724, -0.5506901096452796], 2)]Any[([3.2445719908771724, -0.5506901096452796], 2)]Any[([3.2445719908771724, -0.5506901096452796], 2)]Any[([3.2445719908771724, -0.5506901096452796], 2)]Any[([3.2445719908771724, -0.5506901096452796], 2)]Any[([3.2445719908771724, -0.5506901096452796], 2)]Any[([3.2445719908771724, -0.55069

...

0.5017700649166401], 0)]Any[([4.548072700922636, 0.5017700649166401], 0)]Any[([4.548072700922636, 0.5017700649166401], 0)]Any[([4.548072700922636, 0.5017700649166401], 0)]Any[([4.548072700922636, 0.5017700649166401], 0)]Any[([4.503677782640107, 0.317189009575855], 0)]Any[([4.503677782640107, 0.317189009575855], 0)]Any[([4.503677782640107, 0.317189009575855], 0)]Any[([4.503677782640107, 0.317189009575855], 0)]Any[([4.503677782640107, 0.317189009575855], 0)]Any[([4.503677782640107, 0.317189009575855], 0)]Any[([4.503677782640107, 0.317189009575855], 0)]Any[([4.503677782640107, 0.317189009575855], 0)]Any[([4.503677782640107, 0.317189009575855], 0)]Any[([4.503677782640107, 0.317189009575855], 0)]Any[([4.503677782640107, 0.317189009575855], 0)]Any[([4.503677782640107, 0.317189009575855], 0)]Any[([4.503677782640107, 0.317189009575855], 0)]Any[([4.503677782640107, 0.317189009575855], 0)]Any[([4.503677782640107, 0.317189009575855], 0)]Any[([4.503677782640107, 0.317189009575855], 0)]Any[([4.5036

[([3.9525561618466227, -0.3819328895741488], 0)]Any[([3.9525561618466227, -0.3819328895741488], 0)]Any[([3.9525561618466227, -0.3819328895741488], 0)]Any[([3.9525561618466227, -0.3819328895741488], 0)]Any[([3.9525561618466227, -0.3819328895741488], 0)]Any[([3.9525561618466227, -0.3819328895741488], 0)]Any[([3.9525561618466227, -0.3819328895741488], 0)]Any[([3.9525561618466227, -0.3819328895741488], 0)]Any[([3.9525561618466227, -0.3819328895741488], 0)]Any[([3.9525561618466227, -0.3819328895741488], 0)]Any[([3.9525561618466227, -0.3819328895741488], 0)]Any[([3.9525561618466227, -0.3819328895741488], 0)]Any[([3.9525561618466227, -0.3819328895741488], 0)]Any[([3.9525561618466227, -0.3819328895741488], 0)]Any[([3.9525561618466227, -0.3819328895741488], 0)]Any[([3.9525561618466227, -0.3819328895741488], 0)]Any[([3.9525561618466227, -0.3819328895741488], 0)]Any[([3.9525561618466227, -0.3819328895741488], 0)]Any[([3.9525561618466227, -0.3819328895741488], 0)]Any[([3.9525561618466227, -0.38193

Excessive output truncated after 524291 bytes.

0)]Any[([3.0554580496346113, -0.6708696749613012], 0)]Any[([3.0554580496346113, -0.6708696749613012], 0)]Any[([3.3997997791716315, -0.8803950458655377], 0)]Any[([3.3997997791716315, -0.8803950458655377], 0)]Any[([3.3997997791716315, -0.8803950458655377], 0)]Any[([3.3997997791716315, -0.8803950458655377], 0)]Any[([3.3997997791716315, -0.8803950458655377], 0)]Any[([3.3997997791716315, -0.8803950458655377], 0)]Any[([3.3997997791716315, -0.8803950458655377], 0)]Any[([3.3997997791716315, -0.8803950458655377], 0)]Any[([3.3997997791716315, -0.8803950458655377], 0)]Any[([3.3997997791716315, -0.8803950458655377], 0)]Any[([3.3997997791716315, -0.8803950458655377], 0)]Any[([3.3997997791716315, -0.8803950458655377], 0)]Any[([3.3997997791716315, -0.8803950458655377], 0)]Any[([3.3997997791716315, -0.8803950458655377], 0)]Any[([3.3997997791716315, -0.8803950458655377], 0)]Any[([3.3997997791716315, -0.8803950458655377], 0)]Any[([3.3997997791716315, -0.8803950458655377], 0)]Any[(

LoadError: InterruptException: